In [10]:
import sys
import os
from sklearn.datasets import make_classification
import pandas as pd


In [11]:
# Add the parent directory to sys.path
sys.path.append(os.path.abspath(".."))  

from mcp.multiclasspredict import run_classification

In [12]:
# load the real data

X_raw_1 = pd.read_csv('../input/HP_multiomics/Amplicon_X.csv', sep="\t", index_col=0)
X_raw_2 = pd.read_csv('../input/HP_multiomics/RNA_X.csv', sep="\t", index_col=0)
y_raw = pd.read_csv('../input/HP_multiomics/metadata.csv', sep = "\t", index_col=0)


In [13]:
# make sure X and y have same index
index_intersect = X_raw_1.index.intersection(X_raw_2.index)
index_intersect = index_intersect.intersection(y_raw.index)

X_raw_1 = X_raw_1.loc[index_intersect]
X_raw_2 = X_raw_2.loc[index_intersect]
y_raw = y_raw.loc[index_intersect]

X_raw = pd.concat([X_raw_1, X_raw_2], axis=1)

# y must be a 1D df, select only class information
y = y_raw.loc[:,"Sample Condition"]

# check if index aligns
print((X_raw_1.index == X_raw_2.index).all())
print((X_raw_1.index == X_raw.index).all())
print((X_raw.index == y_raw.index).all())

True
True
True


In [14]:
n_seeds = 3
ks = [10]
result_path = "results/real_data"

run_classification(X_raw_1, y, ks, n_seeds, "rf", None, os.path.join(result_path, f"rf_16s.csv"))
run_classification(X_raw_1, y, ks, n_seeds, "xgb", None, os.path.join(result_path, f"xgb_16s.csv"))
run_classification(X_raw_1, y, ks, n_seeds, "etc", None, os.path.join(result_path, f"etc_16s.csv"))
run_classification(X_raw_1, y, ks, n_seeds, "lgbm", None, os.path.join(result_path, f"lgbm_16s.csv"))

run_classification(X_raw_2, y, ks, n_seeds, "rf", None, os.path.join(result_path, f"rf_trans.csv"))
run_classification(X_raw_2, y, ks, n_seeds, "xgb", None, os.path.join(result_path, f"xgb_trans.csv"))
run_classification(X_raw_2, y, ks, n_seeds, "etc", None, os.path.join(result_path, f"etc_trans.csv"))
run_classification(X_raw_2, y, ks, n_seeds, "lgbm", None, os.path.join(result_path, f"lgbm_trans.csv"))

run_classification(X_raw, y, ks, n_seeds, "rf", None, os.path.join(result_path, f"rf_both.csv"))
run_classification(X_raw, y, ks, n_seeds, "xgb", None, os.path.join(result_path, f"xgb_both.csv"))
run_classification(X_raw, y, ks, n_seeds, "etc", None, os.path.join(result_path, f"etc_both.csv"))
run_classification(X_raw, y, ks, n_seeds, "lgbm", None, os.path.join(result_path, f"lgbm_both.csv"))


[10, 6228]
3
CV for seed 0 and 10 features
Performing One vs Rest classification
AUC for class 'Negative control': 0.5156
AUC for class 'Patient': 0.6432
AUC for class 'Positive control': 0.4824
Macro AUC (OvR): 0.5471
Macro Precision (OvR): 0.4583
Macro Recall (OvR): 0.3486
Macro F1 (OvR): 0.3358
Macro MCC (OvR): 0.0125
Performing One vs One classification


/home/paul/miniconda3/envs/ML/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Macro AUC (OvO): 0.5656
Macro Precision (OvO): 0.3701
Macro Recall (OvO): 0.3218
Macro F1 (OvO): 0.3186
Macro MCC (OvO): 0.0940
{'Negative control vs Rest - AUC': 0.5156084656084656, 'Patient vs Rest - AUC': 0.6431803490627019, 'Positive control vs Rest - AUC': 0.4824238366253766, 'Negative control vs Rest - Precision': 0.675, 'Negative control vs Rest - Recall': 0.9, 'Negative control vs Rest - F1': 0.7714285714285715, 'Negative control vs Rest - MCC': 0.026975697149641292, 'Patient vs Rest - Precision': 0.5, 'Patient vs Rest - Recall': 0.07692307692307693, 'Patient vs Rest - F1': 0.13333333333333333, 'Patient vs Rest - MCC': 0.07406517254704087, 'Positive control vs Rest - Precision': 0.2, 'Positive control vs Rest - Recall': 0.06896551724137931, 'Positive control vs Rest - F1': 0.10256410256410256, 'Positive control vs Rest - MCC': -0.06349888652795518, 'OvR Macro AUC': 0.5470708837655147, 'OvR Macro Precision': 0.4583333333333333, 'OvR Macro Recall': 0.34862953138815206, 'OvR Macro

/home/paul/miniconda3/envs/ML/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/paul/miniconda3/envs/ML/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Macro AUC (OvO): 0.6258
Macro Precision (OvO): 0.2393
Macro Recall (OvO): 0.3218
Macro F1 (OvO): 0.2745
Macro MCC (OvO): 0.0540
{'Negative control vs Rest - AUC': 0.5777777777777778, 'Patient vs Rest - AUC': 0.7388493859082094, 'Positive control vs Rest - AUC': 0.44861064613324403, 'Negative control vs Rest - Precision': 0.6793893129770993, 'Negative control vs Rest - Recall': 0.9888888888888889, 'Negative control vs Rest - F1': 0.8054298642533937, 'Negative control vs Rest - MCC': 0.037265506960015404, 'Patient vs Rest - Precision': 0.0, 'Patient vs Rest - Recall': 0.0, 'Patient vs Rest - F1': 0.0, 'Patient vs Rest - MCC': 0.0, 'Positive control vs Rest - Precision': 0.0, 'Positive control vs Rest - Recall': 0.0, 'Positive control vs Rest - F1': 0.0, 'Positive control vs Rest - MCC': -0.037265506960015404, 'OvR Macro AUC': 0.5884126032730771, 'OvR Macro Precision': 0.22646310432569974, 'OvR Macro Recall': 0.32962962962962966, 'OvR Macro F1': 0.2684766214177979, 'OvR Macro MCC': 0.0, '

...

Macro AUC (OvO): 0.6125
Macro Precision (OvO): 0.2281
Macro Recall (OvO): 0.2989
Macro F1 (OvO): 0.2587
Macro MCC (OvO): -0.0265
{'Negative control vs Rest - AUC': 0.5740740740740741, 'Patient vs Rest - AUC': 0.7420814479638009, 'Positive control vs Rest - AUC': 0.43756277201205224, 'Negative control vs Rest - Precision': 0.6870229007633588, 'Negative control vs Rest - Recall': 1.0, 'Negative control vs Rest - F1': 0.8144796380090498, 'Negative control vs Rest - MCC': -0.05217170974402157, 'Patient vs Rest - Precision': 0.0, 'Patient vs Rest - Recall': 0.0, 'Patient vs Rest - F1': 0.0, 'Patient vs Rest - MCC': 0.05217170974402157, 'Positive control vs Rest - Precision': 0.0, 'Positive control vs Rest - Recall': 0.0, 'Positive control vs Rest - F1': 0.0, 'Positive control vs Rest - MCC': 0.0, 'OvR Macro AUC': 0.5845727646833091, 'OvR Macro Precision': 0.22900763358778628, 'OvR Macro Recall': 0.3333333333333333, 'OvR Macro F1': 0.27149321266968324, 'OvR Macro MCC': 0.0, 'Negative control

/home/paul/miniconda3/envs/ML/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Macro AUC (OvO): 0.5280
Macro Precision (OvO): 0.3276
Macro Recall (OvO): 0.2874
Macro F1 (OvO): 0.2843
Macro MCC (OvO): 0.0171
{'Negative control vs Rest - AUC': 0.5407407407407407, 'Patient vs Rest - AUC': 0.724628312863607, 'Positive control vs Rest - AUC': 0.4047539337127553, 'Negative control vs Rest - Precision': 0.6859504132231405, 'Negative control vs Rest - Recall': 0.9222222222222223, 'Negative control vs Rest - F1': 0.7867298578199052, 'Negative control vs Rest - MCC': -0.01169105365954633, 'Patient vs Rest - Precision': 0.0, 'Patient vs Rest - Recall': 0.0, 'Patient vs Rest - F1': 0.0, 'Patient vs Rest - MCC': -0.013008836323457357, 'Positive control vs Rest - Precision': 0.25, 'Positive control vs Rest - Recall': 0.06896551724137931, 'Positive control vs Rest - F1': 0.10810810810810811, 'Positive control vs Rest - MCC': 0.021667401361211928, 'OvR Macro AUC': 0.5567076624390344, 'OvR Macro Precision': 0.3119834710743802, 'OvR Macro Recall': 0.33039591315453387, 'OvR Macro F

/home/paul/miniconda3/envs/ML/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/paul/miniconda3/envs/ML/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Macro AUC (OvO): 0.6231
Macro Precision (OvO): 0.2222
Macro Recall (OvO): 0.2989
Macro F1 (OvO): 0.2549
Macro MCC (OvO): -0.0744
{'Negative control vs Rest - AUC': 0.6214285714285714, 'Patient vs Rest - AUC': 0.8151260504201681, 'Positive control vs Rest - AUC': 0.46534984934717105, 'Negative control vs Rest - Precision': 0.6793893129770993, 'Negative control vs Rest - Recall': 0.9888888888888889, 'Negative control vs Rest - F1': 0.8054298642533937, 'Negative control vs Rest - MCC': 0.037265506960015404, 'Patient vs Rest - Precision': 0.0, 'Patient vs Rest - Recall': 0.0, 'Patient vs Rest - F1': 0.0, 'Patient vs Rest - MCC': 0.0, 'Positive control vs Rest - Precision': 0.0, 'Positive control vs Rest - Recall': 0.0, 'Positive control vs Rest - F1': 0.0, 'Positive control vs Rest - MCC': -0.037265506960015404, 'OvR Macro AUC': 0.6339681570653035, 'OvR Macro Precision': 0.22646310432569974, 'OvR Macro Recall': 0.32962962962962966, 'OvR Macro F1': 0.2684766214177979, 'OvR Macro MCC': 0.0, 